### Can we classify each phase as relatively low or high anxiety for each subject? ###

In [5]:
# IMPORTING MODULES
import glob
import importlib
import matplotlib.pyplot as plt
import numpy as np
import os
cvx_path = os.path.abspath(os.path.join('..', '..', 'cvxEDA', 'src'))
module_path = os.path.abspath(os.path.join('..', '..', 'src'))
import pandas as pd
import random
import scipy.signal as ss
import shap
import sys
sys.path.append(module_path)

import tools.data_reader_apd as dr
import tools.display_tools as dt
import tools.preprocessing as preprocessing
import train

import lightgbm as lgb
from lightgbm import LGBMClassifier
from scipy.fft import fft, fftfreq, fftshift
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, \
    mean_absolute_error, mean_squared_error, log_loss
from sklearn.model_selection import train_test_split, cross_validate, RepeatedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import normalize
from xgboost import XGBClassifier

import cvxopt.solvers
cvxopt.solvers.options['show_progress'] = False

import warnings
warnings.filterwarnings(
    "ignore", 
    category=RuntimeWarning
)

temp_a, _ = train.Train_APD.get_apd_data_ranking([train.Metrics.BPM], phases=dr.Phases.PHASES_LIST, normalize=False)
idx = temp_a[temp_a["bpm"] > 200].index 
invalid_apd_subjects = set(temp_a["subject"].iloc[idx].tolist())
idx = temp_a[temp_a["bpm"] < 35].index 
invalid_apd_subjects.update(set(temp_a["subject"].iloc[idx].tolist()))

In [10]:
importlib.reload(train)

models = {
    "SVM": SVC(),
    "LGB": LGBMClassifier(),
    "RF": RandomForestClassifier(random_state=16),
    "XGB": XGBClassifier(random_state=16),
    # "random": None
}

parameters = {
    "SVM": [{
        "kernel": ["rbf", "poly", "sigmoid"],
        "C": [0.1, 1, 10, 100],
        "gamma": [1, 0.1, 0.01, 0.001, "scale", "auto"],
    }],
    "LGB": [{
        "objective": ["binary"],
        "num_leaves": [10, 20, 30, 40, 50],
        "max_depth": [3, 4, 5, 6, 7],
        "metric": ["mean_absolute_error", "mean_squared_error", "binary_logloss"]
    }],
    "RF": [{
        "n_estimators": [10, 20, 30, 40, 50],
        "max_features": ["sqrt", "0.4"],
        "min_samples_split": [3, 4, 5, 6, 7],
        "random_state": [16]
    }],
    "XGB": [{
        "objective": ["binary:logistic"],
        "learning_rate": [0.01, 0.1, 0.3, 0.5],
        "max_depth": [4, 5, 6, 7],
        "n_estimators": [10, 20, 30, 40, 50],
        "eval_metric": ["error"],
        "use_label_encoder": [False],
        "random_state": [16]
    }],
    # "random": None
}

metrics = [
    train.Metrics.BPM, 
    train.Metrics.RMSSD, 
    train.Metrics.HF_RR, 
    train.Metrics.LF_RR, 
    train.Metrics.SDNN, 
    # train.Metrics.MEAN_SCL, 
    # train.Metrics.SCR_RATE, 
# ]
] + train.Metrics.STATISTICAL

In [11]:
# K-FOLD CROSS-VALIDATION FOR HYPERPARAMETER SELECTION
importlib.reload(train)
importlib.reload(dr)
importlib.reload(dt)


random.seed(74)

model_phases = [
    "Baseline_Rest", 
    "BugBox_Relax", "BugBox_Anticipate", "BugBox_Exposure", "BugBox_Break",
    "Speech_Relax", "Speech_Anticipate", "Speech_Exposure", "Speech_Break"
]

threshold = "fixed"

anxiety_label_type = None
x, y = train.Train_APD.get_apd_data_ranking(
    metrics, model_phases, verbose=False, anxiety_label_type=anxiety_label_type, 
    threshold=threshold, normalize=True, combine_phases=False, standardize=False
)
x = x.drop(["phaseId"], axis=1)
inds = pd.isnull(x).any(axis=1).to_numpy().nonzero()[0]
x = x.drop(labels=inds, axis=0).reset_index(drop=True)
y = y.drop(labels=inds, axis=0).reset_index(drop=True)

# drop subjects with noisy data
# x = x[~x["subject"].isin(invalid_apd_subjects)].reset_index(drop=True)
# y = y[~y["subject"].isin(invalid_apd_subjects)].reset_index(drop=True)

if anxiety_label_type is not None:
    x.drop(labels=["anxietyGroup"], axis=1)
    
acc_results = {
    "SVM": [],
    "LGB": [],
    "RF": [],
    "XGB": [],
    # "random": []
}
reports = {
    "SVM": [],
    "LGB": [],
    "RF": [],
    "XGB": [],
    # "random": []
}
best_models = {}

num_iters = 5
get_importance = True
for _ in range(num_iters):
    # HYPERPARAMETER TUNING
    model_data = train.grid_search_cv(
        models, parameters, x, y, by_subject=True, save_metrics=True, is_resample=True,
        get_importance=get_importance, drop_subject=True, test_size=0.1, folds=5
    )

    for model_name in models.keys():
        best_models[model_name] = model_data[model_name]["best_model"]
        print(f"{model_name}: {model_data[model_name]['best_params']}")

    # # FEATURE SELECTION
    x_train, y_train = model_data["train"]
    features = {name: metrics for name in models.keys()}
    # features = train.feature_selection(best_models, model_data["cv"], x_train, y_train, n_features=5)

    # TEST USING OPTIMIZED MODELS AND FEATAURES
    x_test, y_test = model_data["test"]
    out = train.train_test_model(best_models, features, x_train, y_train, x_test, y_test)

    for model_name in acc_results:
        acc_results[model_name].append(out[model_name]["performance"][0])
        reports[model_name].append(out[model_name]["performance"][1])
        if get_importance:
            try:
                print("")
                feature_imp = list(zip(metrics + ["lf_hf_ratio"], out[model_name]["performance"][2]))
                feature_imp = sorted(feature_imp, key=lambda x: x[1], reverse=True)
                print(feature_imp)
            except Exception as e:
                print(out[model_name]["performance"][2])
            print("")

for model_name in acc_results.keys():
    print(f"Model evaluation metrics for {model_name}:")
    for i in range(len(reports[model_name])):
        report = reports[model_name][i]
        acc = acc_results[model_name][i]
        p = report["precision"]
        r = report["recall"]
        f1 = report["f1"]
        auc = report["auc"]
        print(f"\tAccuracy: {acc}\n\tPrecision: {p}\n\tRecall: {r}\n\tF1-score: {f1}\n\tAUC score: {auc}\n" + "-"*40)
    print(f"Mean acc: {np.mean([acc_results[model_name][i] for i in range(len(reports[model_name]))])}")
    print(f"Mean F1-score: {np.mean([reports[model_name][i]['f1'] for i in range(len(reports[model_name]))])}")
    print(f"Mean AUC score: {np.mean([reports[model_name][i]['auc'] for i in range(len(reports[model_name]))])}")
    print("\n")

Grid search for SVM ...
Grid search for LGB ...
Grid search for RF ...


One or more of the test scores are non-finite: [0.58274378 0.5994079  0.604087   0.60532733 0.60405462 0.58218584
 0.59258353 0.5996136  0.60209516 0.61059589 0.59713648 0.59795275
 0.60401312 0.60215059 0.61223914 0.58729582 0.59375975 0.59803072
 0.60187976 0.60480357 0.62233236 0.63161699 0.62586954 0.62187762
 0.62796732        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan]


Grid search for XGB ...
SVM: {'C': 100, 'gamma': 'auto', 'kernel': 'sigmoid'}
LGB: {'max_depth': 3, 'metric': 'mean_absolute_error', 'num_leaves': 10, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 7, 'n_estimators': 20, 'random_state': 16}
XGB: {'eval_metric': 'error', 'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 20, 'objective': 'binary:logistic', 'random_state': 16, 'use_label_encoder': False}
Feature selection for SVM ...
Feature selection for LGB ...
Feature selection for RF ...
Feature selection for XGB ...
Training SVM ...
Model SVM, Actual: [0 1], [76 43], Predictions: [0 1], [63 56]
coef_ only available for SVC with linear kernel
Training LGB ...
Model LGB, Actual: [0 1], [76 43], Predictions: [0 1], [81 38]
Training RF ...
Model RF, Actual: [0 1], [76 43], Predictions: [0 1], [86 33]
Training XGB ...
Model XGB, Actual: [0 1], [76 43], Predictions: [0 1], [89 30]

None


[('hf_rr', 135), ('sdnn', 99), ('bpm', 97), ('lf_rr', 93), ('rmssd', 92)

One or more of the test scores are non-finite: [0.54367666 0.56719824 0.56584148 0.5723459  0.57671797 0.55090664
 0.55154109 0.56307802 0.55561297 0.55883921 0.5298998  0.55433987
 0.553341   0.56127116 0.56142809 0.58539324 0.58509894 0.5866581
 0.57863241 0.57924969 0.59274142 0.58157632 0.58481541 0.58417755
 0.5810448         nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan]


Grid search for XGB ...
SVM: {'C': 100, 'gamma': 0.01, 'kernel': 'sigmoid'}
LGB: {'max_depth': 7, 'metric': 'mean_absolute_error', 'num_leaves': 30, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 7, 'n_estimators': 10, 'random_state': 16}
XGB: {'eval_metric': 'error', 'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 20, 'objective': 'binary:logistic', 'random_state': 16, 'use_label_encoder': False}
Feature selection for SVM ...
Feature selection for LGB ...
Feature selection for RF ...
Feature selection for XGB ...
Training SVM ...
Model SVM, Actual: [0 1], [59 57], Predictions: [0 1], [95 21]
coef_ only available for SVC with linear kernel
Training LGB ...
Model LGB, Actual: [0 1], [59 57], Predictions: [0 1], [60 56]
Training RF ...
Model RF, Actual: [0 1], [59 57], Predictions: [0 1], [63 53]
Training XGB ...
Model XGB, Actual: [0 1], [59 57], Predictions: [0 1], [85 31]

None


[('rmssd', 361), ('lf_rr', 359), ('hf_rr', 310), ('bpm', 289), ('sdnn', 23

...

Grid search for XGB ...
SVM: {'C': 100, 'gamma': 'auto', 'kernel': 'sigmoid'}
LGB: {'max_depth': 4, 'metric': 'mean_absolute_error', 'num_leaves': 20, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 7, 'n_estimators': 40, 'random_state': 16}
XGB: {'eval_metric': 'error', 'learning_rate': 0.3, 'max_depth': 5, 'n_estimators': 10, 'objective': 'binary:logistic', 'random_state': 16, 'use_label_encoder': False}
Feature selection for SVM ...
Feature selection for LGB ...
Feature selection for RF ...
Feature selection for XGB ...
Training SVM ...
Model SVM, Actual: [0 1], [64 57], Predictions: [0 1], [67 54]
coef_ only available for SVC with linear kernel
Training LGB ...
Model LGB, Actual: [0 1], [64 57], Predictions: [0 1], [76 45]
Training RF ...
Model RF, Actual: [0 1], [64 57], Predictions: [0 1], [63 58]
Training XGB ...
Model XGB, Actual: [0 1], [64 57], Predictions: [0 1], [77 44]

None


[('hf_rr', 186), ('bpm', 179), ('rmssd', 175), ('lf_rr', 161), ('sdnn', 

One or more of the test scores are non-finite: [0.5906418  0.59653744 0.59550509 0.59932247 0.5911103  0.58012079
 0.56854498 0.5822728  0.58940413 0.59075736 0.5983633  0.58394769
 0.59064245 0.58869317 0.59246341 0.57696145 0.56683746 0.58010323
 0.58310863 0.58239045 0.58060286 0.59127031 0.59492391 0.5908746
 0.58025305        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan]


Grid search for XGB ...
SVM: {'C': 100, 'gamma': 'auto', 'kernel': 'sigmoid'}
LGB: {'max_depth': 7, 'metric': 'mean_absolute_error', 'num_leaves': 10, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 3, 'n_estimators': 40, 'random_state': 16}
XGB: {'eval_metric': 'error', 'learning_rate': 0.3, 'max_depth': 5, 'n_estimators': 20, 'objective': 'binary:logistic', 'random_state': 16, 'use_label_encoder': False}
Feature selection for SVM ...
Feature selection for LGB ...
Feature selection for RF ...
Feature selection for XGB ...
Training SVM ...
Model SVM, Actual: [0 1], [61 58], Predictions: [0 1], [73 46]
coef_ only available for SVC with linear kernel
Training LGB ...
Model LGB, Actual: [0 1], [61 58], Predictions: [0 1], [86 33]
Training RF ...
Model RF, Actual: [0 1], [61 58], Predictions: [0 1], [86 33]
Training XGB ...
Model XGB, Actual: [0 1], [61 58], Predictions: [0 1], [76 43]

None


[('sdnn', 205), ('hf_rr', 200), ('bpm', 178), ('lf_rr', 168), ('rmssd', 

One or more of the test scores are non-finite: [0.61244392 0.61101652 0.61208733 0.61288478 0.61389295 0.60133516
 0.60278441 0.61417762 0.61190964 0.61855927 0.57753404 0.5883442
 0.60191908 0.61342332 0.61858092 0.62000923 0.63056004 0.62771554
 0.63500536 0.63563877 0.5944888  0.60495366 0.60934612 0.62029534
 0.62336877        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan]


Grid search for XGB ...
SVM: {'C': 100, 'gamma': 'auto', 'kernel': 'rbf'}
LGB: {'max_depth': 5, 'metric': 'mean_absolute_error', 'num_leaves': 10, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 6, 'n_estimators': 50, 'random_state': 16}
XGB: {'eval_metric': 'error', 'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 30, 'objective': 'binary:logistic', 'random_state': 16, 'use_label_encoder': False}
Feature selection for SVM ...
Feature selection for LGB ...
Feature selection for RF ...
Feature selection for XGB ...
Training SVM ...
Model SVM, Actual: [0 1], [64 57], Predictions: [0 1], [96 25]
coef_ only available for SVC with linear kernel
Training LGB ...
Model LGB, Actual: [0 1], [64 57], Predictions: [0 1], [75 46]
Training RF ...
Model RF, Actual: [0 1], [64 57], Predictions: [0 1], [75 46]
Training XGB ...
Model XGB, Actual: [0 1], [64 57], Predictions: [0 1], [85 36]

None


[('lf_rr', 181), ('sdnn', 177), ('bpm', 166), ('hf_rr', 151), ('rmssd', 149)

In [12]:
# ENSEMBLE
importlib.reload(train)
importlib.reload(dr)
importlib.reload(dt)

from sklearn.ensemble import VotingClassifier
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score

test_size = 0.15
folds = 5
ensemble_models = [
    "SVM", 
    "LGB",
    "RF",
    "XGB"
]

cv, x_train, y_train, x_test, y_test = train.kfold_train_test_split(x, y, test_size, is_resample=False, folds=folds)
y_train = y_train.loc[:, "label"]
y_test = y_test.loc[:, "label"]

estimators = [(name, best_models[name]) for name in ensemble_models]
ensemble = VotingClassifier(estimators, voting="hard")
ensemble.fit(x_train, y_train)
y_pred = ensemble.predict(x_test)

acc = accuracy_score(y_test, y_pred)
p = precision_score(y_test, y_pred, zero_division=0)
r = recall_score(y_test, y_pred, zero_division=0)
f1 = f1_score(y_test, y_pred, zero_division=0)
auc = roc_auc_score(y_test, y_pred)
print(f"\tAccuracy: {acc}\n\tPrecision: {p}\n\tRecall: {r}\n\tF1-score: {f1}\n\tAUC score: {auc}\n" + "-"*40)

	Accuracy: 0.5806451612903226
	Precision: 0.5365853658536586
	Recall: 0.3235294117647059
	F1-score: 0.4036697247706422
	AUC score: 0.5525693035835023
----------------------------------------
